In [1]:
!pip install -q lxml 

# Librairies

import bs4 
import lxml
import pandas as pd 
import urllib

from urllib import request


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


# 1. Recupération des plus gros succcès au box-office

In [11]:
url_bo = "https://fr.wikipedia.org/wiki/Liste_des_plus_gros_succ%C3%A8s_du_box-office_mondial"
request_text = request.urlopen(url_bo).read()


In [12]:
# Recherche des balises HTML
page = bs4.BeautifulSoup(request_text, "lxml")

In [16]:
tableau_bo = page.find('table', {'class', "wikitable sortable centre"})
print(tableau_bo)

<table class="wikitable sortable centre">
<caption><b>Liste des 50 plus gros succès du box-office mondial</b><sup class="reference" id="cite_ref-11"><a href="#cite_note-11"><span class="cite_crochet">[</span>a 1<span class="cite_crochet">]</span></a></sup>
</caption>
<tbody><tr>
<th scope="col">Rang
</th>
<th scope="col">Meilleur<br/>rang
</th>
<th scope="col">Titre
</th>
<th scope="col">Réalisateur
</th>
<th scope="col">Année
</th>
<th scope="col">Pays de<br/>production
</th>
<th scope="col">Recettes
</th></tr>
<tr>
<td>1
</td>
<td>1
</td>
<td><i><a href="/wiki/Avatar_(film,_2009)" title="Avatar (film, 2009)">Avatar</a></i>
</td>
<td><a href="/wiki/James_Cameron" title="James Cameron">James Cameron</a>
</td>
<td align="center"><a href="/wiki/2009_au_cin%C3%A9ma" title="2009 au cinéma">2009</a>
</td>
<td><span class="nowrap"><span class="datasortkey" data-sort-value="États unis"><span class="flagicon"><span class="mw-image-border noviewer" typeof="mw:File"><a class="mw-file-description

In [17]:
# Recupération de chaque ligne du tableau
table_body = tableau_bo.find('tbody')
rows = table_body.find_all('tr')

In [19]:
# Nettoyage de la couche HTML pour ne garder qye le texte utile sur une ligne
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    print(cols)

[]
['1', '1', 'Avatar', 'James Cameron', '2009', 'États-Unis', '2\xa0923\xa0706\xa0026\xa0$[a 2]']
['2', '1', 'Avengers: Endgame', 'Anthony etJoe Russo', '2019', 'États-Unis', '2\xa0799\xa0439\xa0100\xa0$[a 3]']
['3', '3', "Avatar\xa0: La Voie de l'eau", 'James Cameron', '2022', 'États-Unis', '2\xa0320\xa0250\xa0281\xa0$[a 4]']
['4', '1', 'Titanic', 'James Cameron', '1997', 'États-Unis', '2\xa0264\xa0743\xa0305\xa0$[a 5]']
['5', '3', 'Star Wars, épisode VII\xa0: Le Réveil de la Force', 'J. J. Abrams', '2015', 'États-Unis', '2\xa0071\xa0310\xa0218\xa0$[a 6]']
['6', '4', 'Avengers: Infinity War', 'Anthony etJoe Russo', '2018', 'États-Unis', '2\xa0052\xa0415\xa0039\xa0$[a 7]']
['7', '6', 'Spider-Man: No Way Home', 'Jon Watts', '2021', 'États-Unis', '1\xa0921\xa0847\xa0111\xa0$[a 8]']
['8', '3', 'Jurassic World', 'Colin Trevorrow', '2015', 'États-Unis', '1\xa0671\xa0537\xa0444\xa0$[a 9]']
['9', '7', 'Le Roi lion', 'Jon Favreau', '2019', 'États-Unis', '1\xa0663\xa0075\xa0401\xa0$[a 10]']
['

In [22]:
# Création d'un dictionnaire avec les infos obtenues pour en faire un DataFrame

dico_bo = dict()
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    if len(cols) > 0 : 
        dico_bo[cols[0]] = cols[1:]
dico_bo

{'1': ['1',
  'Avatar',
  'James Cameron',
  '2009',
  'États-Unis',
  '2\xa0923\xa0706\xa0026\xa0$[a 2]'],
 '2': ['1',
  'Avengers: Endgame',
  'Anthony etJoe Russo',
  '2019',
  'États-Unis',
  '2\xa0799\xa0439\xa0100\xa0$[a 3]'],
 '3': ['3',
  "Avatar\xa0: La Voie de l'eau",
  'James Cameron',
  '2022',
  'États-Unis',
  '2\xa0320\xa0250\xa0281\xa0$[a 4]'],
 '4': ['1',
  'Titanic',
  'James Cameron',
  '1997',
  'États-Unis',
  '2\xa0264\xa0743\xa0305\xa0$[a 5]'],
 '5': ['3',
  'Star Wars, épisode VII\xa0: Le Réveil de la Force',
  'J. J. Abrams',
  '2015',
  'États-Unis',
  '2\xa0071\xa0310\xa0218\xa0$[a 6]'],
 '6': ['4',
  'Avengers: Infinity War',
  'Anthony etJoe Russo',
  '2018',
  'États-Unis',
  '2\xa0052\xa0415\xa0039\xa0$[a 7]'],
 '7': ['6',
  'Spider-Man: No Way Home',
  'Jon Watts',
  '2021',
  'États-Unis',
  '1\xa0921\xa0847\xa0111\xa0$[a 8]'],
 '8': ['3',
  'Jurassic World',
  'Colin Trevorrow',
  '2015',
  'États-Unis',
  '1\xa0671\xa0537\xa0444\xa0$[a 9]'],
 '9': ['7

In [23]:
data_bo = pd.DataFrame.from_dict(dico_bo, orient = 'index')
data_bo.head()

,0,1,2,3,4,5
1,1,Avatar,James Cameron,2009,États-Unis,2 923 706 026 $[a 2]
2,1,Avengers: Endgame,Anthony etJoe Russo,2019,États-Unis,2 799 439 100 $[a 3]
3,3,Avatar : La Voie de l'eau,James Cameron,2022,États-Unis,2 320 250 281 $[a 4]
4,1,Titanic,James Cameron,1997,États-Unis,2 264 743 305 $[a 5]
5,3,"Star Wars, épisode VII : Le Réveil de la Force",J. J. Abrams,2015,États-Unis,2 071 310 218 $[a 6]


In [26]:
# Récupértion des en-têtes de tableau

for row in rows:
    cols = row.find_all('th')
    if len(cols) > 0:
        cols = [ele.get_text(separator= ' ').strip().title() for ele in cols]
        columns_bo = cols
print(columns_bo)

['Rang', 'Meilleur Rang', 'Titre', 'Réalisateur', 'Année', 'Pays De Production', 'Recettes']


In [27]:
data_bo.columns = columns_bo[1:]
data_bo.head()

,Meilleur Rang,Titre,Réalisateur,Année,Pays De Production,Recettes
1,1,Avatar,James Cameron,2009,États-Unis,2 923 706 026 $[a 2]
2,1,Avengers: Endgame,Anthony etJoe Russo,2019,États-Unis,2 799 439 100 $[a 3]
3,3,Avatar : La Voie de l'eau,James Cameron,2022,États-Unis,2 320 250 281 $[a 4]
4,1,Titanic,James Cameron,1997,États-Unis,2 264 743 305 $[a 5]
5,3,"Star Wars, épisode VII : Le Réveil de la Force",J. J. Abrams,2015,États-Unis,2 071 310 218 $[a 6]


In [42]:
# Nettoyage de la colonne Recettes pou rne garder que le revenu brut
for i in range(len(data_bo['Recettes'])):
    data_bo['Recettes'][i] = data_bo['Recettes'][i][:15]
data_bo.head()


,Meilleur Rang,Titre,Réalisateur,Année,Pays De Production,Recettes
1,1,Avatar,James Cameron,2009,États-Unis,2 923 706 026 $
2,1,Avengers: Endgame,Anthony etJoe Russo,2019,États-Unis,2 799 439 100 $
3,3,Avatar : La Voie de l'eau,James Cameron,2022,États-Unis,2 320 250 281 $
4,1,Titanic,James Cameron,1997,États-Unis,2 264 743 305 $
5,3,"Star Wars, épisode VII : Le Réveil de la Force",J. J. Abrams,2015,États-Unis,2 071 310 218 $


### Idées pour brève description des films en présence
- comparer CA en prenant en compte inflation
- Sur représentation d'un réalisateur, pays origine
- nb d'oscar par film 
- corrélation entre bonnes critiques et nb oscar
- différence entre critique populaire et critique spécialisées ? 

In [45]:
data_bo = data_bo[['Titre', 'Réalisateur', 'Année', 'Pays De Production', 'Recettes', 'Meilleur Rang']]

In [46]:
data_bo.head()

,Titre,Réalisateur,Année,Pays De Production,Recettes,Meilleur Rang
1,Avatar,James Cameron,2009,États-Unis,2 923 706 026 $,1
2,Avengers: Endgame,Anthony etJoe Russo,2019,États-Unis,2 799 439 100 $,1
3,Avatar : La Voie de l'eau,James Cameron,2022,États-Unis,2 320 250 281 $,3
4,Titanic,James Cameron,1997,États-Unis,2 264 743 305 $,1
5,"Star Wars, épisode VII : Le Réveil de la Force",J. J. Abrams,2015,États-Unis,2 071 310 218 $,3


In [54]:
data_bo['Réalisateur'].nunique()

39

In [64]:
data_bo[data_bo['Réalisateur'] == 'J. J. Abrams']

,Titre,Réalisateur,Année,Pays De Production,Recettes,Meilleur Rang
5,"Star Wars, épisode VII : Le Réveil de la Force",J. J. Abrams,2015,États-Unis,2 071 310 218 $,3
38,"Star Wars, épisode IX : L'Ascension de Skywalker",J. J. Abrams,2019,États-Unis,1 077 022 372 $,32


# 2. Analyse rapide des données 

# 3. Récupération des commentaires

# 4. NLP et analyse des commentaires 